# Artificial Neural Networks

Martin Kersner

Inspired by http://neuralnetworksanddeeplearning.com/ and https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/src/network.py.

* stochastic gradient descent learning
* backpropagation
* forward pass
* backward pass

In [3]:
from __future__ import division
import random
import numpy as np
from utils import *
import matplotlib.pyplot as plt

%matplotlib inline

## Perceptron

<img src="files/perceptron.png" />

In [4]:
# PERCEPTRON - MATRIX MULTIPLICATION EXAMPLE

# w_1 = 1
# w_2 = 1.1
# w_3 = 0.1
# w_4 = 0.2
W = np.matrix([[1.0, 1.1, 0.1, 0.2]])

# x_1 = 0.5
# x_2 = 0.7
# x_3 = 3
# x_4 = 1.0
X = np.matrix([[0.5],
               [0.7],
               [3],
               [1.0]])

# bias
b = 1

# forward pass
h = np.dot(W, X) + b
print(h)

# Activation function
# a = activation_function(h)

[[ 2.77]]


## Multilayer perceptron

<img src="files/multi-layer-perceptron.png" />

In [10]:
# MULTILAYER PERCEPTRON - MATRIX MULTIPLICATION EXAMPLE

# w_11 = 1.0
# w_12 = 1.1
# w_13 = 1.3
# w_21 = 0.1
# w_22 = 0.2
# w_23 = 0.6
# w_31 = 0.4
# w_32 = 1.2
# w_32 = 1.9
W = np.matrix([[1.0, 1.1, 1.3], 
               [0.1, 0.2, 0.6],
               [0.4, 1.2, 1.9]])

# x_1 = 0.5
# x_2 = 0.7
# x_3 = 0.6
X = np.matrix([[0.5],
               [0.7],
               [0.6]])

#bias
b = np.matrix([[1.0],
               [1.0],
               [1.0]])


# forward pass
h = np.dot(W, X) + b
print(h)

# Activation function
# a = activation_function(h)

[[ 3.05]
 [ 1.55]
 [ 3.18]]


In [ ]:
# DEFINE AND INITIALIZE

# The first layer containing 2 neurons, the second layer 3 neurons, and the third layer 1 neuron.  
layers = [784, 30, 10]

# Altogether 3 layers.
num_layers = len(layers)

# The biases and weights for the network are initialized randomly, using a Gaussian distribution
# with mean 0, and variance 1. The first layer is assumed to be an input layer, and by convention we
# won't set any biases for those neurons.
biases  = [np.random.randn(y, 1) for y in layers[1:]]
weights = [np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])]

In [ ]:
# BIASES in layers
for b in biases:
    print(len(b))

In [ ]:
# BIAS VALUES
print(biases)

In [ ]:
# WEIGHTS in layers
for w in weights:
    print(w.shape)

In [ ]:
# WEIGHTS
print(weights)

## Sigmoid activation function
* introduces non-linearity

<img src="files/sigmoid.png">

In [ ]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

In [ ]:
def feedforward(biases, weights, a):
    for b, w in zip(biases, weights):
        a = sigmoid(np.dot(w, a)+b)
    return a

### Backpropagation

### Derivation of sigmoid

In [ ]:
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

def cost_derivative(output_activations, y):
    """Return the vector of partial derivatives \partial C_x /
    \partial a for the output activations."""
    return (output_activations-y)
        
def backprop(biases, weights, x, y):
    """Return a tuple ``(nabla_b, nabla_w)`` representing the
    gradient for the cost function C_x.  ``nabla_b`` and
    ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
    to ``self.biases`` and ``self.weights``."""
    
    nabla_b = zeros_like(biases)
    nabla_w = zeros_like(weights)
    
    # feedforward
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    
    for b, w in zip(biases, weights):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
        
    # backward pass
    delta = cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    # Note that the variable l in the loop below is used a little
    # differently to the notation in Chapter 2 of the book.  Here,
    # l = 1 means the last layer of neurons, l = 2 is the
    # second-last layer, and so on.  It's a renumbering of the
    # scheme in the book, used here to take advantage of the fact
    # that Python can use negative indices in lists.
    num_layers = len(weights)+1
    for l in xrange(2, num_layers):
        z = zs[-l]
        sp = sigmoid_prime(z)
        delta = np.dot(weights[-l+1].transpose(), delta) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
    return (nabla_b, nabla_w)

## Minibatch
* TODO minibatch explanation
* TOOD update update explanation
* TODO eta explanation

In [ ]:
# eta - learning rate
def update_mini_batch(biases, weights, mini_batch, eta):
    def update(v, nv, eta, mini_batch_size):
        return v-(eta/mini_batch_size)*nv
    
    def cumulate(nabla_v, delta_nabla_v):
        return [nv+dnv for nv, dnv in zip(nabla_v, delta_nabla_v)]
        
    nabla_b = zeros_like(biases)
    nabla_w = zeros_like(weights)
    
    # !! TODO explanation
    for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = backprop(biases, weights, x, y)
        nabla_b = cumulate(nabla_b, delta_nabla_b)
        nabla_w = cumulate(nabla_w, delta_nabla_w)
    
    mini_batch_size = len(mini_batch)
    
    weights = [update(w, nw, eta, mini_batch_size)
               for w, nw in zip(weights, nabla_w)]
    
    biases = [update(b, nb, eta, mini_batch_size)
              for b, nb in zip(biases, nabla_b)]
    
    return biases, weights

* Stochastic gradient descent
* Epoch

In [ ]:
def SGD(biases, weights, training_data, epochs, mini_batch_size, eta, test_data=None):
    """Train the neural network using mini-batch stochastic
    gradient descent.  The ``training_data`` is a list of tuples
    ``(x, y)`` representing the training inputs and the desired
    outputs.  The other non-optional parameters are
    self-explanatory.  If ``test_data`` is provided then the
    network will be evaluated against the test data after each
    epoch, and partial progress printed out.  This is useful for
    tracking progress, but slows things down substantially."""
    
    def evaluate(biases, weights, data, training_data_evalution=False):
        if training_data_evalution:
            test_results = [(np.argmax(feedforward(biases, weights, x)), np.argmax(y)) for (x, y) in data]
        else:
            test_results = [(np.argmax(feedforward(biases, weights, x)), y) for (x, y) in data]

        return sum(int(x == y) for (x, y) in test_results)
    
    if test_data:
        n_test = len(test_data)

    n = len(training_data)
    test_acc_lst = []
    train_acc_lst = []
    
    for j in xrange(epochs):
        random.shuffle(training_data)
        
        mini_batches = [training_data[k:k+mini_batch_size] for k in xrange(0, n, mini_batch_size)]
        
        for mini_batch in mini_batches:
            biases, weights = update_mini_batch(biases, weights, mini_batch, eta)
            
        if test_data:
            test_acc = evaluate(biases, weights, test_data)
            test_acc_lst.append(test_acc/n_test)
            
            train_acc = evaluate(biases, weights, training_data, training_data_evalution=True)
            train_acc_lst.append(train_acc/n)
            
            print("Epoch {0}: {1} / {2}".format(j, test_acc, n_test))
        else:
            print("Epoch {0} complete".format(j))
            
    return biases, weights, train_acc_lst, test_acc_lst

In [ ]:
# def evaluate(biases, weights, test_data):
#     """Return the number of test inputs for which the neural
#     network outputs the correct result. Note that the neural
#     network's output is assumed to be the index of whichever
#     neuron in the final layer has the highest activation."""
    
#     test_results = [(np.argmax(feedforward(biases, weights, x)), y) for (x, y) in test_data]
#     return sum(int(x == y) for (x, y) in test_results)

## MNIST dataset
* http://yann.lecun.com/exdb/mnist/
* 28 by 28 pixels
* grayscale images

In [ ]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [ ]:
# Display random image and label from training data.

def random_data_sample(data):
    ri = random.randint(0, len(data)-1)
    X_rand = data[ri][0]
    y_rand = data[ri][1]
    
    return X_rand, y_rand

def display_random_digit(data):
    X_rand, y_rand = random_data_sample(data)

    rand_img = X_rand.reshape((28, 28))
    plt.figure()
    plt.imshow(rand_img, cmap='gray')
    
    print(np.argmax(y_rand))
    
display_random_digit(training_data)

## One hot encoding

In [ ]:
def display_random_one_hot_encoding(data):
    _, y_rand = random_data_sample(data)
    print(y_rand)
    plt.figure()
    plt.imshow(y_rand.T, cmap='gray')
    print(np.argmax(y_rand))
    
display_random_one_hot_encoding(training_data)

TODO display random numbers
TODO graph

In [ ]:
%time

# Training
epochs=3
mini_batch_size=10
eta=3.0

biases_final, weights_final, train_acc, test_acc = SGD(biases, weights, training_data, epochs, mini_batch_size, eta, test_data=validation_data)

# Plot training curves
plt.plot(range(len(test_acc)), test_acc, c="r")
plt.plot(range(len(train_acc)), train_acc, c="b")
plt.ylabel("accuracy")
plt.xlabel("epochs")

## Keras